In [66]:
import cryptocompare
import pandas as pd
import datetime
import boto3
import datetime
import requests
import json
from functools import reduce
import matplotlib.pyplot as plt
import numpy as np
import re
%matplotlib inline
from datetime import datetime, timedelta
import awswrangler as wr


CRYPTO_COMPARE_KEY = "6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4"
cryptocompare.cryptocompare._set_api_key_parameter(CRYPTO_COMPARE_KEY)

'&api_key=6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4'

In [2]:
session = boto3.Session(profile_name='master')
s3_client = session.client('s3')
bucket = 'kosmobiker-masterproject'

In [102]:
def get_list_of_objects_s3(operation_parameters, s3=None):
    """
    List files in specific S3 URL
    """
    if not s3:
        s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(**operation_parameters)
    for page in page_iterator:
        for content in page.get('Contents'):
            if content.get('LastModified') > utc.localize(datetime.now() - timedelta(days=4)):
                yield content.get('Key')


In [103]:
def get_json_s3(bucket, key, s3=None):
    """
    Function is used to get JSON file
    from S3 bucket
    """
    if not s3:
        s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8')
    return json.loads(data)

In [104]:
params = {
        'Bucket' : bucket,
        'Prefix' : "data/daily_crypto_data/"
        }

In [109]:
data = []
for key in get_list_of_objects_s3(params, s3_client):
    try:
        if key.endswith('.json'):
            coin_cur = re.findall("(?:[0-9]{4}_[0-9]{2}_[0-9]{2})_(.+)(?:.json)", key)[0]
            tmp = get_json_s3(bucket, key, s3_client)
            res = [dict(item, **{'coin_currency':f'{coin_cur}'}) for item in tmp]
            data.extend(res)
    except Exception as err:
        print(err)
#write to glue table

In [110]:
pd.DataFrame(data)

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,coin_currency
0,1658247600,0.01149,0.01147,0.01148,141.66,1.6250,0.01149,direct,,BNB_BTC
1,1658247660,0.01149,0.01148,0.01149,254.89,2.9230,0.01148,direct,,BNB_BTC
2,1658247720,0.01148,0.01146,0.01148,494.91,5.6680,0.01146,direct,,BNB_BTC
3,1658247780,0.01146,0.01145,0.01146,77.17,0.8833,0.01145,direct,,BNB_BTC
4,1658247840,0.01146,0.01144,0.01145,257.65,2.9470,0.01144,direct,,BNB_BTC
...,...,...,...,...,...,...,...,...,...,...
224791,1658592960,0.35590,0.35550,0.35550,167153.23,59429.5600,0.35590,direct,,XRP_USD
224792,1658593020,0.35600,0.35590,0.35590,19342.01,6882.5100,0.35590,direct,,XRP_USD
224793,1658593080,0.35590,0.35580,0.35590,14548.59,5176.5000,0.35580,direct,,XRP_USD
224794,1658593140,0.35590,0.35570,0.35580,171008.17,60829.2600,0.35580,direct,,XRP_USD


In [81]:
def write_to_glue(pandas_df: pd.DataFrame, path_table, database_name, table_name):
    try:
        wr.s3.to_parquet(
            df=pandas_df,
            index=False,
            path=path_table,
            dataset=True,
            database=database_name,
            compression='snappy',
            table=table_name,
            mode="append",
            # partition_cols=["coin"],
            use_threads=True,
            concurrent_partitioning=True,
            boto3_session=session
        )
    except Exception as err:
        print(err)

In [78]:
wr.catalog.databases(boto3_session=session)


,Database,Description
0,crypto_data,OHLC data + Tweets
1,default,Default Hive database
2,master_project,this is a test database


In [79]:
df = pd.DataFrame(data)

In [84]:
# write_to_glue(df, path_table=f"s3://{bucket}/data/crypto_data/", database_name="crypto_data", table_name='daily_ohlc')

In [85]:
params_tweets = {
        'Bucket' : bucket,
        'Prefix' : "data/daily_tweets/"
        }

In [100]:
import pytz

utc=pytz.UTC

def get_latest_tweets(operation_parameters, s3=None):
    """
    List files in specific S3 URL
    """
    if not s3:
        s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(**operation_parameters)
    for page in page_iterator:
        for content in page.get('Contents'):
            if content.get('LastModified') > utc.localize(datetime.now() - timedelta(days=4)):
                yield content.get('Key')

In [111]:
tweets = []
for key in get_latest_tweets(params_tweets, s3_client):
    tmp = get_json_s3(bucket, key, s3_client)
    tweets.extend(tmp)


In [112]:
pd.DataFrame(tweets)

,id,created_at,ceil_datetime,month,weekday,day,hour,minute,QUERY,ticker,text,favorite_count,result_type,user_name,followers_count,retweet_count
0,1549486114490179590,2022-07-19 20:07:32,2022-07-19 20:15:00,7,2,19,20,15,bitcoin,BTC,"In Q2 2022, #Bitcoin mining efficiency surged ...",3410,popular,saylor,2632470,683
1,1549125925022928898,2022-07-18 20:16:16,2022-07-18 20:30:00,7,1,18,20,30,bitcoin,BTC,Facebook's former Head of Messenger: #Bitcoin ...,3401,popular,BitcoinMagazine,2572534,667
2,1549099043153928192,2022-07-18 18:29:27,2022-07-18 18:30:00,7,1,18,18,30,bitcoin,BTC,"Former $10 trillion BlackRock manager: ""#Bitco...",4414,popular,BitcoinMagazine,2572534,795
3,1549544628097089538,2022-07-20 00:00:03,2022-07-20 00:15:00,7,3,20,0,15,bitcoin,BTC,https://t.co/t6kVcSK6IH\n\nCrypto market:\n$XE...,0,recent,CryptoTreeMap,75,0
4,1549544627664916480,2022-07-20 00:00:03,2022-07-20 00:15:00,7,3,20,0,15,bitcoin,BTC,$BTCUSD ALERT: 24hr volume was 73.53% higher t...,0,recent,btcusdalerts,1197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37058,1550182467654496256,2022-07-21 18:14:35,2022-07-21 18:15:00,7,4,21,18,15,Litecoin,LTC,@litecoin @BinanceAcademy Altcoins again? Can ...,1,recent,MdAtiqu18884986,35,0
37059,1550182403737518081,2022-07-21 18:14:20,2022-07-21 18:15:00,7,4,21,18,15,Litecoin,LTC,Litecoin mining 10 times slower than a couple ...,0,recent,Programmatoris,527,0
37060,1550182368547426307,2022-07-21 18:14:12,2022-07-21 18:15:00,7,4,21,18,15,Litecoin,LTC,RT @PhibbiX: 250$ Litecoin Giveaway \n5 Winner...,0,recent,gabriellcristea,274,220
37061,1550182341468860416,2022-07-21 18:14:05,2022-07-21 18:15:00,7,4,21,18,15,Litecoin,LTC,RT @coin_gabbar: #CoinGabbar Prediction:\n#LIT...,0,recent,rajputana_jayu,8,61
